<a href="https://colab.research.google.com/github/Bmiguez/Arezzo-Valuation/blob/main/Valuation_Arezzo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Importando inicialmente as bibliotecas necessárias
import pandas as pd
import numpy as np
import statsmodels.api as sm
import yfinance as yf
import sys
import datetime
from dateutil.relativedelta import relativedelta

In [ ]:
# Definindo ticker e índice de referencia
ticker = "ARZZ3.SA"
indice = "^BVSP"

# Definindo a data de referência e o intervalo de dados
ano = 2023
mes = 1
dia = 1

data_ref = datetime.date(ano, mes, dia)
data_ini = data_ref - relativedelta(months = 46)

In [ ]:
from ast import increment_lineno
# Extrair retornos ajustados da empresa e do benchmark

retornos = yf.download([ticker, indice], start = data_ini, end = data_ref, interval="1mo")["Adj Close"].pct_change()

retornos = pd.DataFrame(retornos.dropna())

y = retornos[ticker]
x = retornos[indice]



[*********************100%***********************]  2 of 2 completed


In [ ]:
# Começando a regressão

x = sm.add_constant(x)

modelo = sm.OLS(y, x)

resultados = modelo.fit()

print(resultados.summary())
# Extraindo valor do Beta

beta = resultados.params[1]

conf_interval = resultados.conf_int(alpha=0.05)

lower_bound = conf_interval.iloc[1, 0]
upper_bound = conf_interval.iloc[1, 1]

print("Intervalo de Confiança:")
print("Limite Inferior:", lower_bound)
print("Limite Superior:", upper_bound)


                            OLS Regression Results                            
Dep. Variable:               ARZZ3.SA   R-squared:                       0.551
Model:                            OLS   Adj. R-squared:                  0.541
Method:                 Least Squares   F-statistic:                     52.80
Date:                Thu, 15 Jun 2023   Prob (F-statistic):           5.30e-09
Time:                        01:25:23   Log-Likelihood:                 55.153
No. Observations:                  45   AIC:                            -106.3
Df Residuals:                      43   BIC:                            -102.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0106      0.011      0.978      0.3

In [ ]:
# Cálculo do Custo do equity

# Desalavancando beta

tax = 0.27
debt = 40000
equity = 60000

unlev_beta = beta/(1+(1-tax)*debt/equity)

# Risk Free Obtido 14/06
rf = 10.96%

# Default spread 14/06
dfs = 3.14%



# Calculando equity premium

data_ini = data_ref - relativedelta(years = 5)

retornos = yf.download(indice, start = data_ini, end = data_ref, interval="1mo")["Adj Close"].pct_change()

retornos = retornos.dropna()

retorno_anualizado = (float(retornos.mean())+1)**12-1
print(retorno_anualizado)

[*********************100%***********************]  1 of 1 completed
0.08755353683124478


In [4]:
path = "https://github.com/Bmiguez/Arezzo-Valuation/raw/main/Dados_Valuation.xlsx"
aba_bp = "BP"
aba_dre = "DRE"
aba_fc = "FC"
aba_lp = "LojasProp"
aba_indicadores = "Indicadores"
aba_receitabruta = "Receita Bruta"

#### Criação dos Dataframes
#### Balanço Patrimonial
df_bp = pd.read_excel(path, aba_bp)

#### Demonstração Resultado do Exercício
df_dre = pd.read_excel(path, aba_dre)

#### Fluxo de Caixa
df_fc = pd.read_excel(path, aba_fc)

#### Lojas Próprias
df_lp = pd.read_excel(path, aba_lp)

#### Indicadores
df_ind = pd.read_excel(path, aba_indicadores)

#### Receita Bruta
df_receita = pd.read_excel(path, aba_receitabruta)


,Ano,Número de pares vendidos ('000),Número de bolsas vendidas ('000),Número de peças de roupas vendidas ('000)²,Número de funcionários,Número de lojas*,Próprias,Franquias,Outsourcing Calçados e Bolsas (% da produção total),SSS¹ sell-in (franquias),SSS¹ sell-out (lojas próprias + web + franquias)
0,2011,7532.803000,473.434000,NaN,1879,341,45,296,0.863000,NaN,0.000000
1,2012,8980.299000,552.131000,NaN,2058,399,57,342,0.885000,0.122000,0.000000
2,2013,9890.636000,636.850000,NaN,1946,458,55,403,0.911000,0.020000,0.023000
3,2014,10621.805000,827.081000,NaN,2091,516,54,462,0.900000,0.040000,0.080000
4,2015,10418.339510,897.004000,NaN,2106,543,49,494,0.914000,-0.088000,-0.016000
5,2016,11180.815142,1097.077870,NaN,2233,565,50,515,0.900000,0.043000,0.041000
6,2017,12132.000000,1257.000000,NaN,2405,618,50,568,0.909000,0.050000,0.039000
7,2018,13506.646306,1519.993465,NaN,2437,685,51,634,0.915538,0.047096,0.041514
8,2019,14204.586531,1723.778119,NaN,2465,752,53,699,0.906630,0.016513,0.039009
9,2020,13031.854548,1373.720344,934.521807,2260,901,139,762,0.905535,-0.251902,-0.238000
